In [35]:
# Imports
import random

import math

class Vertex:
    def __init__(self, key, x, y):
        self.key = key
        self.x = x
        self.y = y
        self.d = float('-inf')  # Distance from source
        self.h = None  # Heuristic distance to destination
        self.pi = None
        self.pi_set = set()
        self.priority = 0
        self.parent = None

    def __lt__(self, other):
        return self.priority < other.priority

    def __gt__(self, other):
        return self.priority > other.priority


class Graph:
    def __init__(self):
        self.vertices = {}
        self.adjacency_list = {}

    def add_vertex(self, key, x, y):
        if key not in self.vertices:
            self.vertices[key] = Vertex(key, x, y)
            self.adjacency_list[key] = []

    def add_edge(self, u, v):
        if u in self.vertices and v in self.vertices and u != v:
            if v not in self.adjacency_list[u]:
                self.adjacency_list[u].append(v)
            if u not in self.adjacency_list[v]:
                self.adjacency_list[v].append(u)  # For undirected graph


    def get_adjacency_list(self, vertex_key):
        if vertex_key in self.vertices:
            return self.adjacency_list[vertex_key]
        else:
            return []
    
    def max_degree(self):
        max_degree = 0
        for vertex_key in self.vertices.keys():
            degree = len(self.adjacency_list[vertex_key])
            max_degree = max(max_degree, degree)
        return max_degree
    
    def average_degree(self):
        total_degree = 0
        for vertex_key in self.vertices.keys():
            total_degree += len(self.adjacency_list[vertex_key])
        if len(self.vertices) > 0:
            return total_degree / len(self.vertices)
        else:
            return 0

    def generate_geometric_graph(self, n, r):

        for key in range(n):
            self.add_vertex(key,random.uniform(0, 1), random.uniform(0, 1))

        for u in self.vertices:
            for v in self.vertices:
                if u != v and euclidean_distance(self.vertices[u], self.vertices[v]) <= r:
                    self.adjacency_list[u].append(v)
                    self.adjacency_list[v].append(u)


def euclidean_distance(v1, v2):
    return math.sqrt(((v1.x - v2.x) ** 2) + ((v1.y - v2.y) ** 2))



n_values = [300, 400, 500]
min_ratios = [0.9, 0.8, 0.7]
max_ratios = [0.95, 0.9, 0.8]

graph = Graph()

def largest_connected_component(graph):
    def dfs(graph, visited, vertex, component):
        visited.add(vertex)
        component.append(vertex)
        for neighbor in graph.get_adjacency_list(vertex):
            if neighbor not in visited:
                dfs(graph, visited, neighbor, component)

    visited = set()
    largest_component = []
    for vertex in graph.vertices:
        if vertex not in visited:
            component = []
            dfs(graph, visited, vertex, component)
            if len(component) > len(largest_component):
                largest_component = component

    lcc_graph = Graph()
    for vertex_key in largest_component:
        vertex = graph.vertices[vertex_key]
        lcc_graph.add_vertex(vertex.key, vertex.x, vertex.y)
        for neighbor_key in graph.get_adjacency_list(vertex_key):
            if neighbor_key in largest_component:
                lcc_graph.add_edge(vertex.key, neighbor_key)

    return lcc_graph


def binary_search_r(n, min_ratio, max_ratio):
    low = 0.0
    high = 1.0
    while low <= high:
        graph = Graph()
        mid = (low + high) / 2
        vlcc_min = int(min_ratio * n)
        vlcc_max = int(max_ratio * n)
        graph.generate_geometric_graph(n, mid)
        lcc = largest_connected_component(graph)
        lcc_size = len(lcc.vertices)
        if vlcc_min <= lcc_size <= vlcc_max:
            return mid, lcc
        elif lcc_size > vlcc_min:
            high = mid - 0.00001
        else:
            low = mid + 0.00001
    return None, None  # No suitable r found


def store_graph_to_file(graph, filename):
    with open(filename, 'w') as f:
        processed_edges = set()  # To avoid duplicates
        for u in graph.vertices:
            for v in graph.get_adjacency_list(u):
                # Generate a unique key for the edge (u, v) by sorting the vertices' keys
                edge_key = (u, v) if u < v else (v, u)
                if edge_key not in processed_edges:
                    vertex_u = graph.vertices[u]
                    vertex_v = graph.vertices[v]
                    f.write(f"{u} {vertex_u.x} {vertex_u.y} {v} {vertex_v.x} {vertex_v.y}\n")
                    processed_edges.add(edge_key)


n_values = [300, 400, 500]
min_ratios = [0.9, 0.8, 0.7]
max_ratios = [0.95, 0.9, 0.8]

# for n, min_ratio, max_ratio in zip(n_values, min_ratios, max_ratios):
#     r, lcc = binary_search_r(n, min_ratio, max_ratio)
#     if r is not None:
#         store_graph_to_file(lcc, f"graph_n{n}_r{r}.EDGES")
#         print(f"Graph with n={n} and r={r} stored successfully.")
#     else:
#         print(f"No suitable r found for n={n}.")


def read_graph_from_file(filename):
    graph = Graph()
    with open(filename, 'r') as f:
        for line in f:
            data = line.split()
            u = int(data[0])
            x1 = float(data[1])
            y1 = float(data[2])
            v = int(data[3])
            x2 = float(data[4])
            y2 = float(data[5])

            if u not in graph.vertices:
                graph.add_vertex(u, x1, y1)
            if v not in graph.vertices:
                graph.add_vertex(v, x2, y2)

            graph.add_edge(u, v)

    return graph


# graph = read_graph_from_file("graph_n300_r0.08202289062500001.EDGES")
graph = read_graph_from_file("graph_n400_r0.06249125.EDGES")
# graph = read_graph_from_file("graph_n500_r0.05467859375.EDGES")

if 13 in graph.vertices:
    print("yes")


yes


In [18]:
len(graph.vertices)

369

In [29]:
#  A star
import heapq

def initialize_single_source_max(graph, source, destination):
    for vertex in graph.vertices.values():
        vertex.d = float('-inf')
        vertex.h = euclidean_distance(vertex, destination)
        vertex.pi = None
        vertex.pi_set = set()

    source.d = 0

def relax_max(u, v):
    if v.d < (u.d + 1):
        v.d = u.d + 1
        v.pi = u
        return True
    return False

def a_star_max(graph, source, destination):
    initialize_single_source_max(graph, source, destination)
    closed_set = set()
    open_heap = []
    heapq.heappush(open_heap, (-source.d - source.h, source))
    
    while open_heap:
        _, u = heapq.heappop(open_heap)
        closed_set.add(u)

        if u == destination:
            break

        for v_key in graph.get_adjacency_list(u.key):
            v = graph.vertices[v_key]
            if v in closed_set:
                continue
            if relax_max(u, v):
                v.priority = v.d + v.h
                heapq.heappush(open_heap, (-v.priority, v))

final_path = []

for i in range(0, len(graph.vertices)):
    if i in graph.vertices:
        for j in range(0, len(graph.vertices)):
            if i != j and j in graph.vertices:
                s = graph.vertices[i]
                d = graph.vertices[j]
                a_star_max(graph, s, d)
                path = []
                x = d
                if x.d != float('-inf'):
                    visited = {x}
                    while x != s:
                        path.insert(0, x.key)
                        visited.add(x)
                        x = x.pi
                        if x in visited:
                            break
                    path.insert(0, s.key)
                    if (len(path) > len(final_path)):
                        final_path = path

final_path

                

[178,
 90,
 275,
 76,
 132,
 278,
 207,
 230,
 189,
 64,
 292,
 126,
 17,
 23,
 222,
 285,
 137,
 224,
 150,
 24,
 214,
 155,
 235,
 205,
 252,
 101,
 121,
 288,
 255,
 180,
 160,
 106,
 227,
 84,
 282,
 165,
 264,
 139,
 53,
 212,
 5,
 236,
 123,
 25,
 182,
 256,
 63,
 228,
 98,
 11,
 77,
 38,
 263,
 268,
 105,
 193,
 72,
 110,
 188,
 20,
 55,
 158,
 6,
 173,
 35,
 129,
 8,
 130,
 151,
 237,
 262,
 217,
 290,
 161,
 168,
 284,
 12,
 147,
 149,
 167,
 118,
 127,
 170,
 108,
 14,
 297,
 184,
 163,
 148,
 95,
 112,
 143,
 238,
 89,
 223,
 94,
 156,
 86,
 159,
 45,
 266,
 277,
 28,
 211,
 117,
 88,
 125,
 59,
 107,
 225,
 140,
 194,
 100,
 259,
 289,
 253,
 31,
 36,
 243,
 215,
 287,
 181,
 274,
 213,
 294,
 281,
 15,
 251,
 240,
 21,
 47,
 7,
 152,
 61,
 226,
 144,
 206,
 103,
 208,
 34,
 102,
 183,
 299,
 56,
 83,
 296,
 234,
 279,
 291,
 185,
 229,
 44,
 16,
 166,
 81,
 254,
 113,
 115,
 62,
 248,
 242,
 3,
 92,
 85,
 190,
 49,
 65,
 96,
 271,
 265,
 249,
 202,
 283,
 136,
 231,
 241,


In [36]:
# ADEEL
def DFSAlgo(graph):
    def dfs(node, visited, path, memo):
        nonlocal max_length, longest_path
        visited[node] = True
        path.append(node)

        for neighbor in graph.adjacency_list[node]:
            if not visited[neighbor]:
                if neighbor not in memo or len(memo[neighbor]) < len(path):
                    dfs(neighbor, visited, path, memo)

        if len(path) > max_length:
            max_length = len(path)
            longest_path = path.copy()

        path.pop()
        visited[node] = False
        memo[node] = path.copy()

    max_length = float('-inf')
    longest_path = []

    visited = {v: False for v in graph.vertices}
    memo = {}

    for vertex in graph.vertices:
        if not visited[vertex]:
            dfs(vertex, visited, [], memo)

    return longest_path


DFSAlgo(graph)

[305,
 269,
 161,
 179,
 38,
 223,
 399,
 26,
 210,
 323,
 390,
 155,
 169,
 348,
 218,
 189,
 0,
 322,
 45,
 94,
 253,
 247,
 4,
 67,
 2,
 89,
 142,
 198,
 316,
 69,
 59,
 71,
 153,
 158,
 175,
 315,
 202,
 137,
 36,
 314,
 358,
 19,
 126,
 102,
 277,
 241,
 50,
 53,
 162,
 58,
 10,
 27,
 32,
 265,
 229,
 55,
 65,
 47,
 20,
 11,
 124,
 257,
 347,
 88,
 107,
 68,
 57,
 74,
 177,
 335,
 354,
 40,
 85,
 92,
 381,
 237,
 101,
 127,
 99,
 77,
 225,
 160,
 13,
 29,
 33,
 39,
 97,
 64,
 150,
 172,
 192,
 220,
 73,
 123,
 222,
 246,
 309,
 296,
 340,
 294,
 379,
 186,
 113,
 62,
 7,
 46,
 63,
 163,
 80,
 51,
 37,
 297,
 111,
 307,
 255,
 215,
 280,
 333,
 109,
 81,
 374]

In [37]:
def DFSAlgo(graph):
    def dfs(node, visited, path, memo):
        nonlocal max_length, longest_path
        visited[node] = True
        path.append(node)

        for neighbor in graph.adjacency_list[node]:
            if not visited[neighbor]:
                if neighbor not in memo:
                    dfs(neighbor, visited, path, memo)
                elif memo[neighbor] is not None:
                    if len(path) + len(memo[neighbor]) > max_length:
                        max_length = len(path) + len(memo[neighbor])
                        longest_path = path + memo[neighbor]

        path.pop()
        visited[node] = False
        memo[node] = longest_path[len(path):]

    max_length = float('-inf')
    longest_path = []
    memo = {v: None for v in graph.vertices}
    visited = {v: False for v in graph.vertices}

    for vertex in graph.vertices:
        dfs(vertex, visited, [], memo)

    return longest_path


DFSAlgo(graph)

[190,
 66,
 371,
 388,
 187,
 195,
 302,
 82,
 385,
 239,
 191,
 96,
 130,
 271,
 128,
 312,
 233,
 90,
 104,
 304,
 194,
 56,
 293,
 41,
 217,
 282,
 249,
 140,
 48,
 365,
 291,
 24,
 258,
 311,
 21,
 5,
 393,
 334,
 327,
 76,
 70,
 329,
 87,
 289,
 389,
 193,
 110,
 242,
 98,
 196,
 18,
 375,
 372,
 147,
 118,
 83,
 259,
 251,
 121,
 272,
 336,
 308,
 275,
 157,
 3,
 362,
 318,
 353,
 234,
 352,
 303,
 270,
 248,
 260,
 211,
 176,
 299,
 356,
 290,
 100,
 317,
 1,
 252,
 168,
 149,
 300,
 205,
 203,
 185,
 129,
 380,
 346,
 374,
 81,
 132,
 360,
 333,
 343,
 280,
 324,
 255,
 307,
 306,
 131,
 325,
 37,
 51,
 283,
 119,
 80,
 387,
 46,
 7,
 63,
 222,
 123,
 338,
 301,
 246,
 73,
 220,
 296,
 309,
 192,
 150,
 396,
 113,
 285,
 97,
 39,
 33,
 29,
 13,
 294,
 340,
 172,
 392,
 225,
 377,
 319,
 183,
 204,
 127,
 101,
 398,
 381,
 92,
 85,
 91,
 40,
 354,
 57,
 335,
 177,
 74,
 68,
 107,
 391,
 347,
 257,
 238,
 141,
 171,
 159,
 395,
 47,
 383,
 331,
 279,
 65,
 55,
 229,
 265,
 32,
 2

In [34]:
def DFSAlgo(graph):
    def dfs(node, visited, path, memo):
        nonlocal max_length, longest_path
        visited[node] = True
        path.append(node)

        for neighbor in graph.adjacency_list[node]:
            if not visited[neighbor]:
                if neighbor not in memo:
                    dfs(neighbor, visited, path, memo)
                elif memo[neighbor] is not None:
                    if len(path) + len(memo[neighbor]) > max_length:
                        max_length = len(path) + len(memo[neighbor])
                        longest_path = path + memo[neighbor]

        path.pop()
        visited[node] = False
        memo[node] = longest_path[len(path):]

    max_length = float('-inf')
    longest_path = []
    memo = {v: None for v in graph.vertices}
    visited = {v: False for v in graph.vertices}

    for vertex in graph.vertices:
        dfs(vertex, visited, [], memo)

    return longest_path

# Assuming graph is an object of a class with adjacency_list and vertices attributes
# graph = YourGraphClass()  # Replace YourGraphClass with the actual class name
result = DFSAlgo(graph)

DFSAlgo(graph)

[283,
 289,
 274,
 181,
 71,
 263,
 297,
 108,
 14,
 127,
 118,
 284,
 149,
 168,
 147,
 170,
 60,
 290,
 114,
 87,
 217,
 223,
 94,
 89,
 159,
 45,
 28,
 277,
 266,
 237,
 151,
 86,
 130,
 8,
 129,
 35,
 195,
 173,
 158,
 55,
 203,
 20,
 67,
 135,
 51,
 110,
 268,
 105,
 193,
 139,
 84,
 282,
 264,
 165,
 106,
 160,
 180,
 288,
 121,
 101,
 252,
 255,
 205,
 123,
 5,
 236,
 25,
 235,
 132,
 76,
 278,
 275,
 207,
 230,
 272,
 232,
 64,
 189,
 69,
 292,
 285,
 221,
 222,
 137,
 126,
 17,
 224,
 75,
 256,
 155,
 214,
 228,
 63,
 24,
 150,
 98,
 66,
 201,
 162,
 93,
 82,
 192,
 220,
 52,
 164,
 78,
 109,
 171,
 204,
 104,
 246,
 138,
 41,
 50,
 270,
 276,
 97,
 43,
 120,
 293,
 191,
 286,
 273,
 79,
 46,
 128,
 245,
 175,
 146,
 29,
 103,
 206,
 144,
 7,
 226,
 211,
 61,
 117,
 88,
 184,
 95,
 148,
 143,
 125,
 179,
 259,
 253,
 294,
 287,
 281,
 243,
 194,
 215,
 140,
 100,
 36,
 31,
 251,
 240,
 208,
 56,
 34,
 47,
 234,
 176,
 81,
 199,
 99,
 54,
 18,
 269,
 248,
 242,
 92,
 3,
 190,
 

In [ ]:
# def DFSLongestPath(graph):
#     def dfs(node, path, memo):
#         if node in memo:
#             return memo[node]
        
#         longest = path + [node]
#         for neighbor in graph.adjacency_list[node]:
#             if neighbor not in path:
#                 candidate = dfs(neighbor, path + [node], memo)
#                 if len(candidate) > len(longest):
#                     longest = candidate

#         memo[node] = longest
#         return longest

#     memo = {}
#     longest_path = []

#     for vertex in graph.vertices:
#         current_path = dfs(vertex, [], memo)
#         if len(current_path) > len(longest_path):
#             longest_path = current_path

#     return longest_path

# # Example usage:
# longest_path = DFSLongestPath(graph)
# # print("Longest path:", sorted(longest_path))
# longest_path